In [19]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import fetch_lfw_people
from sklearn.datasets import fetch_olivetti_faces
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
import matplotlib.pyplot as plt
import cv2
import random
from skimage.color import rgb2gray
import pickle
import os
dirname = os.path.abspath('')
datasets_path = os.path.join(dirname, 'Datasets')
people_path = os.path.join(datasets_path, 'Our faces')
parameters_path = os.path.join(dirname, 'PCA Parameters\Our faces')

In [14]:
def load_our_dataset():
    #  Loop over every directory in people path
    y = dict()
    images = []
    i=0
    min_width = float('inf')
    min_height = float('inf')
    
    for directory in os.listdir(people_path):
        #  Loop over every image in the directory
        for image in os.listdir(os.path.join(people_path, directory)):
            #  Load the image
            img = plt.imread(os.path.join(people_path, directory, image))
            img = rgb2gray(img)
            # resize image to width*height
            if (np.max(img) > 1):
                img = img / 255.0
            
            if(img.shape[0] < min_height):
                min_height = img.shape[0]
            if(img.shape[1] < min_width):
                min_width = img.shape[1]
                
            images.append(img)
            y[i] = [directory]
            i+=1
    
    images = np.array([cv2.resize(image, (100, 100))for image in images])
        
    return images, y

In [15]:
train_images, y = load_our_dataset()
print(train_images.shape)
print(len(y))

(32, 100, 100)
32


## PCA

In [20]:
def extract_pca_features(images, load=False, num_pca_components=0.95):
        image_vectors = []
        for image in images:
            image_vectors.append(image.flatten())
        image_vectors = np.array(image_vectors)
        
        if load:
            pca = pickle.load(open(parameters_path + "\our_faces_pca.pkl", "rb"))
            pca_features = pca.transform(image_vectors)
            return pca_features
        else:
            print("Creating new PCA model...")
            pca = PCA(n_components = num_pca_components, svd_solver = 'full')
            pca.fit(image_vectors)

            pca_features = pca.transform(image_vectors)

            pca_features = np.array(pca_features)

            pickle.dump(pca, open(parameters_path + "\our_faces_pca.pkl", "wb"))
            
            return pca_features

In [21]:
pca_features_train = extract_pca_features(train_images,load=False, num_pca_components=0.95)
print(f'PCA Features Train: {pca_features_train.shape}')

Creating new PCA model...
PCA Features Train: (32, 15)
